In [1]:
import matplotlib.pyplot as plt
import numpy as np
from darts.utils import timeseries_generation as tg
from darts.models import TransformerModel

[2021-10-11 11:33:19,281] WARNING | darts.models | Support for AutoARIMA is not available. To enable it, install u8darts[pmdarima] or u8darts[all].
[2021-10-11 11:33:19,283] WARNING | darts.models | Support Facebook Prophet is not available. To enable it, install u8darts[prophet] or u8darts[all].
[2021-10-11 11:33:20,228] WARNING | darts.models | Support for LightGBM not available. To enable LightGBM support in Darts, follow the detailed install instructions for LightGBM in the README: https://github.com/unit8co/darts/blob/master/README.md


In [2]:
LENGTH = 3 * 365  # 3年的逐日数据

# Melting: 具有年周期性和加性白噪声的正弦
melting = 0.9 * tg.sine_timeseries(
    length=LENGTH, value_frequency=(1 / 365), freq="D", column_name="melting"
) + 0.1 * tg.gaussian_timeseries(length=LENGTH, freq="D")

# Rainfalls: 具有双周周期和加性白噪声的正弦信号
rainfalls = 0.5 * tg.sine_timeseries(
    length=LENGTH, value_frequency=(1 / 14), freq="D", column_name="rainfall"
) + 0.5 * tg.gaussian_timeseries(length=LENGTH, freq="D")

# 我们将`Melting`提前 5 天
melting_contribution = 0.5 * melting.shift(5)

# 我们为rainfalls计算的类似分布
all_contributions = [melting_contribution] + [
    0.1 * rainfalls.shift(lag) for lag in range(5)
]

# 我们将最终流量计算为所有内容的总和；
# 修剪series，使它们都具有相同的开始时间
flow = sum(
    [
        series[melting_contribution.start_time() :][: melting.end_time()]
        for series in all_contributions
    ]
).with_columns_renamed("melting", "flow") 

# 加入一些白噪声
flow += 0.1 * tg.gaussian_timeseries(length=len(flow)) #  (time: 1090, component: 1, sample: 1)
flow_train, _ = flow.split_before(0.8)

In [3]:
flow_train

<TimeSeries (DataArray) (time: 871, component: 1, sample: 1)>
array([[[ 0.21565815]],

       [[ 0.21423436]],

       [[ 0.06957858]],

       [[-0.10584966]],

       [[-0.05372487]],

       [[-0.01527358]],

       [[-0.11877873]],

       [[-0.20302514]],

       [[-0.45839975]],

       [[-0.341923  ]],

...

       [[ 0.1999118 ]],

       [[ 0.07129746]],

       [[ 0.06521758]],

       [[ 0.0077577 ]],

       [[-0.10689064]],

       [[ 0.19305507]],

       [[ 0.44222566]],

       [[ 0.46928453]],

       [[ 0.46014526]],

       [[ 0.55757766]]])
Coordinates:
  * time       (time) datetime64[ns] 2000-01-06 2000-01-07 ... 2002-05-25
  * component  (component) object 'flow'
Dimensions without coordinates: sample

In [4]:
model = TransformerModel(
    input_chunk_length=24*4*4,    # 输入的时间序列长度
    output_chunk_length=24*4,   # 预测的时间序列长度
)

In [7]:
flow_train.add_datetime_attribute(attribute="week")

<TimeSeries (DataArray) (time: 871, component: 2, sample: 1)>
array([[[ 0.21565815],
        [ 1.        ]],

       [[ 0.21423436],
        [ 1.        ]],

       [[ 0.06957858],
        [ 1.        ]],

       ...,

       [[ 0.46928453],
        [21.        ]],

       [[ 0.46014526],
        [21.        ]],

       [[ 0.55757766],
        [21.        ]]])
Coordinates:
  * time       (time) datetime64[ns] 2000-01-06 2000-01-07 ... 2002-05-25
  * component  (component) object 'flow' 'week'
Dimensions without coordinates: sample

In [5]:
model.fit(
    series=flow_train,
    past_covariates=melting.stack(rainfalls),  # time: 1095，component: 2，sample: 1的xr.dataarray
    epochs=100,
    verbose=True,
)

[2021-10-11 11:33:20,998] INFO | darts.models.forecasting.torch_forecasting_model | Train dataset contains 392 samples.
[2021-10-11 11:33:20,998] INFO | darts.models.forecasting.torch_forecasting_model | Train dataset contains 392 samples.
[2021-10-11 11:33:21,017] INFO | darts.models.forecasting.torch_forecasting_model | Time series values are 64-bits; casting model to float64. If training is too slow you can try casting your data to 32-bits.
[2021-10-11 11:33:21,017] INFO | darts.models.forecasting.torch_forecasting_model | Time series values are 64-bits; casting model to float64. If training is too slow you can try casting your data to 32-bits.


<class 'numpy.ndarray'>


  0%|          | 0/100 [00:00<?, ?it/s]

KeyboardInterrupt: 